In [ ]:
from keras.models import load_model

In [ ]:
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D,Dropout,Flatten
from keras.layers import Dense
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!unzip  'eye_dataset.zip'

unzip:  cannot find or open eye_dataset.zip, eye_dataset.zip.zip or eye_dataset.zip.ZIP.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
training_dir = 'eye_dataset/training'
validation_dir = 'eye_dataset/validation'

In [ ]:
train_datagen = ImageDataGenerator(
			rescale=1./255,
			shear_range=0.2,
			horizontal_flip=True 
		)

val_datagen = ImageDataGenerator(
			rescale=1./255,
			shear_range=0.2,
			horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
	    directory=training_dir,
	    batch_size=32,
      target_size=(24, 24),
	    color_mode="grayscale",
	    class_mode="binary",
	    shuffle=True)

val_generator = val_datagen.flow_from_directory(
	    directory=validation_dir,
	    batch_size=32,
      target_size=(24, 24),
	    color_mode="grayscale",
	    class_mode="binary",
	    shuffle=True)

Found 3779 images belonging to 2 classes.
Found 1067 images belonging to 2 classes.


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
patience =20

#log_file_path = base_path + dataset_name + '_emotion_training.log'
#csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = 'trained_model_5'
model_names = trained_models_path + '.{epoch:02d}.{val_acc:02f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [ ]:
model = Sequential()

model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 16)        1168      
_________________________________________________________________
average_pooling2d_1 (Average (None, 10, 10, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
average_pooling2d_2 (Average (None, 4, 4, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 128)         0         
__________

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit_generator(generator=train_generator,
	                    steps_per_epoch=250,
	                    validation_data=val_generator,
	                    validation_steps=64,
	                    epochs=100,
                      callbacks= callbacks)

Epoch 1/100
250/250 [==============================] - 17s 69ms/step - loss: 0.4768 - acc: 0.7621 - val_loss: 0.3123 - val_acc: 0.8689

Epoch 00001: val_loss did not improve from 0.08514
Epoch 2/100
250/250 [==============================] - 16s 65ms/step - loss: 0.2832 - acc: 0.8909 - val_loss: 0.2318 - val_acc: 0.9192

Epoch 00002: val_loss did not improve from 0.08514
Epoch 3/100
250/250 [==============================] - 16s 65ms/step - loss: 0.2300 - acc: 0.9056 - val_loss: 0.1754 - val_acc: 0.9282

Epoch 00003: val_loss did not improve from 0.08514
Epoch 4/100
250/250 [==============================] - 16s 65ms/step - loss: 0.1722 - acc: 0.9322 - val_loss: 0.1858 - val_acc: 0.9265

Epoch 00004: val_loss did not improve from 0.08514
Epoch 5/100
250/250 [==============================] - 16s 65ms/step - loss: 0.1616 - acc: 0.9350 - val_loss: 0.2119 - val_acc: 0.9182

Epoch 00005: val_loss did not improve from 0.08514
Epoch 6/100
250/250 [==============================] - 16s 66ms/s

In [ ]:
import matplotlib.pyplot as plt
def plotter(trained_record):
    # Loss Curves
    plt.figure(figsize=[8,6])
    plt.plot(trained_record.history['loss'],'r',linewidth=3.0)
    plt.plot(trained_record.history['val_loss'],'b',linewidth=3.0)
    plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.title('Loss Curves',fontsize=16)
    plt.savefig('loss.png')
    # Accuracy Curves
    plt.figure(figsize=[8,6])
    plt.plot(trained_record.history['acc'],'r',linewidth=3.0)
    plt.plot(trained_record.history['val_acc'],'b',linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Accuracy',fontsize=16)
    plt.title('Accuracy Curves',fontsize=16)
    plt.savefig('accuracy.png')

In [ ]:
model_json = model.to_json()
with open("model_5.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model5.h5")